## 25.2 Backtracking

You can apply backtracking to solve a problem with these properties:

1. It's a constraint satisfaction or optimisation problem.
1. Each solution is a set or a sequence of items.
1. The candidates can be constructed incrementally, item by item.
1. Each solution must satisfy local constraints that can be checked as each item is added.

In addition, the way backtracking is presented in M269,
you must know in advance the extensions, i.e.
the items that can be added to a candidate.
Moreover, if a solution is a sequence, items should be unique, but
it may possible to work around this limitation.
For example, we can solve the TSP with backtracking even though
one node appears twice in the tour.

If candidates can't be generated incrementally or if
there are no local constraints, use brute-force search instead.

The following subsections present the code templates
for constraint satisfaction and optimisation problems on sequences and sets.

### 25.2.1 Constraints on sequences

If the problem asks for all sequences that satisfy some constraints,
consider the following:

- What are the candidates? What do they represent?
- What are the items in the candidates? What do items represent?
- What data structure should be used for each item?
- Is the initial candidate the empty sequence?
- What is the initial set of extensions? How can it be generated?
- When is a candidate a solution, i.e. what are the constraints?
- Which constraints are local (can be checked on partial candidates) and
  which are global (must be checked on complete candidates)?
- Can partial candidates be solutions, like in [Trackword](../22_Backtracking/22_3_trackword.ipynb#22.3-Trackword)?

Then copy the following code template and adapt it to your problem,
according to your answers to the above questions.
```python
def problem(instance: object) -> list:
    """Return all solutions for the problem instance, in the order generated."""
    candidate = [...]       # initial candidate, usually []
    extensions = ...        # a set of items
    solutions = []
    extend(candidate, extensions, instance, solutions)
    return solutions

def extend(candidate: list, extensions: set, instance: object, solutions: list) -> None:
    """Add to solutions all extensions of candidate that solve the problem instance."""
    # print('Visiting node', candidate, extensions)
    # remove the next line if partial candidates can be solutions
    if len(extensions) == 0:
        if satisfies_global(candidate, instance):
            solutions.append(candidate)
    for item in extensions:
        if can_extend(item, candidate, instance):
            extend(candidate + [item], extensions - {item}, instance, solutions)

def satisfies_global(candidate: list, instance: object) -> bool:
    """Check if candidate satisfies the global constraints."""
    return ...

def can_extend(item: object, candidate: list, instance: object) -> bool:
    """Check if item may extend candidate towards a solution."""
    return ...
```
In this and the following templates, you may uncomment the print statements to
debug your code and check if the search space is being pruned as you'd expect.
Once your code passes the tests, make the variable names and docstrings
specific to the problem and remove unnecessary parameters.

### 25.2.2 Best sequence

If the problem asks for one sequence that maximises or minimises some value,
consider in addition to the previous questions:

- Is it a minimisation or maximisation problem?
- What value is being minimised or minimised?
- Is there an easily generated 'good' solution, i.e. with a low (or high) value?

The following assumes the value is an integer, but an optimisation problem
can be about any type of comparable values.
If they aren't comparable, it's impossible to determine the best solution.
```python
SOLUTION = 0
VALUE = 1

def problem(instance: object) -> list:
    """Return the best solution the problem instance and its value."""
    candidate = [...]   # initial candidate, usually []
    extensions = ...    # a set of items
    solution = ...      # ideally with a value near lowest/highest
    best = [solution, value(solution, instance)]
    extend(candidate, extensions, instance, best)
    return best

# in the next line replace 'int' by the value's type
def value(candidate: list, instance: object) -> int:
    """Return the value of the candidate sequence for the problem instance."""
    return ...

def extend(candidate: list, extensions: set, instance: object, best: list) -> None:
    """Update best if candidate is a better solution, then extend it."""
    # print('Visiting node', candidate, extensions)
    # remove the next line if partial candidates can be solutions
    if len(extensions) == 0:
        if satisfies_global(candidate, instance):
            candidate_value = value(candidate, instance)
            # in the next line, use < for minimisation problems
            if candidate_value > best[VALUE]:
                # print('New best with value', candidate_value)
                best[SOLUTION] = candidate
                best[VALUE] = candidate_value
    for item in extensions:
        if can_extend(item, candidate, instance):
            extend(candidate + [item], extensions - {item}, instance, best)

def satisfies_global(candidate: list, instance: object) -> bool:
    """Check if candidate satisfies the global constraints."""
    return ...

def can_extend(item: object, candidate: list, instance: object) -> bool:
    """Check if item may extend candidate towards a solution."""
    return ...
```
If no initial solution can be easily constructed, use a pseudo-solution
with an artificial positive or negative infinite value.
The first solution found by backtracking will be the first best solution.
The main function becomes:
```python
import math

def problem(instance: object) -> list:
    """Return the best solution the problem instance and its value."""
    candidate = [...]       # initial candidate, usually []
    extensions = ...        # a set of items
    best = [[], math.inf]   # use -math.inf for maximisation problems
    extend(candidate, extensions, instance, best)
    return best
```
To further prune the search space, consider:

- Does extending a candidate worsen its value, i.e. does extending
  increase the value for a minimisation problem or
  decrease the value for a maximisation problem?

For the TSP, if weights are positive then
extending a candidate path increases its length,
which worsens the candidate's value because we're looking for a shortest tour.
For problems like this, use the next `can_extend` template and
add the `best` parameter to the call in function `extend`.
```python
def can_extend(item: object, candidate: list, instance: object, best: list) -> bool:
    """Check if item can extend candidate into a better solution than best."""
    # replace ... with a check for the local constraints
    # use < for a minimisation problem
    return ... and value(candidate + [item]) > best[VALUE]
```


### 25.2.3 Constraints on sets

If the problem asks for all sets that satisfy some constraints,
use the next template. The questions to consider are the same
as for constraint problems on sequences, except that now
the candidates are sets and the extensions form a sequence.
```python
def problem(instance: object) -> list:
    """Return all solutions for the problem instance, in the order generated."""
    candidate = {...}   # initial candidate, usually set()
    extensions = ...    # a list of items
    solutions = []
    extend(candidate, extensions, instance, solutions)
    return solutions

def extend(candidate: set, extensions: list, instance: object, solutions: list) -> None:
    """Add to solutions all extensions of candidate that solve the problem instance."""
    # print('Visiting node', candidate, extensions)
    if len(extensions) == 0:
        if satisfies_global(candidate, instance):
            solutions.append(candidate)
    else:
        item = extensions[0]
        rest = extensions[1:]
        if can_extend(item, candidate, instance, solutions):    # add item
            extend(candidate.union({item}), rest, instance, solutions)
        extend(candidate, rest, instance, solutions)            # skip item

def satisfies_global(candidate: set, instance: object) -> bool:
    """Check if candidate satisfies the global constraints."""
    return ...

def can_extend(item: object, candidate: set, instance: object) -> bool:
    """Check if item may extend candidate towards a solution."""
    return ...

```
If partial candidates can be solutions, then change the `extend` function to:
```python
    # print('Visiting node', candidate, extensions)
    if satisfies_global(candidate, instance):
        solutions.append(candidate)
    if len(extensions) > 0:
        item = extensions[0]
```


### 25.2.4 Best set

If the problem asks for one set that maximises or minimises some value,
use this template. Consider the same questions as for the best sequence.
```python
SOLUTION = 0
VALUE = 1

def problem(instance: object) -> list:
    """Return the best solution the problem instance and its value."""
    candidate = {...}   # initial candidate, usually set()
    extensions = ...    # a list of items
    solution = ...      # ideally with a value near lowest/highest
    best = [solution, value(solution)]
    extend(candidate, extensions, instance, best)
    return best

def extend(candidate: set, extensions: list, instance: object, best: list) -> None:
    """Update best if candidate is a better solution, then try to extend it."""
    # print('Visiting node', candidate, extensions)
    if len(extensions) == 0:
        if satisfies_global(candidate, instance):
            candidate_value = value(candidate, instance)
            # use < for a minimisation problem
            if candidate_value > best[VALUE]:
                # print('New best with value', candidate_value)
                best[SOLUTION] = candidate
                best[VALUE] = candidate_value
    else:
        item = extensions[0]
        rest = extensions[1:]
        if can_extend(item, candidate, instance, best):
            extend(candidate.union({item}), rest, instance, best)
        extend(candidate, rest, instance, best)
```
As shown earlier, if partial candidates can be solutions then
change the `extend` function, and
if the initial best solution can't be easily constructed,
start with a pseudo-solution of positive or negative infinite value.

To further prune the search space, consider:

- Is it possible to order the extensions by 'incompatibility', i.e. so that
  if an item can’t extend a candidate, none of the following can?

For the knapsack problem we can order items by ascending weight:
if one item doesn't fit the knapsack, nor do the subsequent heavier items.
For problems like this, sort the extensions in the main function and
use this `extend` function.
```python
def extend(candidate: set, extensions: list, instance: object, best: list) -> None:
    """Update best if candidate is a better solution, then try to extend it."""
    # print('Visiting node', candidate, extensions)
    if satisfies_global(candidate, instance):
        candidate_value = value(candidate, instance)
        # use < for a minimisation problem
        if candidate_value > best[VALUE]:
            # print('New best with value', candidate_value)
            best[SOLUTION] = candidate
            best[VALUE] = candidate_value
    if len(extensions) > 0:
        item = extensions[0]
        rest = extensions[1:]
        if can_extend(item, candidate, instance, best):
            extend(candidate.union({item}), rest, instance, best)
            extend(candidate, rest, instance, best)
```

⟵ [Previous section](25_1_graphs.ipynb) | [Up](25-introduction.ipynb) | [Next section](25_3_dp.ipynb) ⟶